# 快速入门 Blackhole SQL功能
----
Blackhole SQL是一款面向大数据的、旨在提供对结构化数据使用SQL语句进行查询、分析、统计等功能的单机计算引擎，提供了数据导入、导出能力，和Blackhole的其他两大模块DataFrame和ML能够无缝对接。  
## Blackhole环境准备
**CodeLab平台默认不安装Blackhole，请先到导航左边“包管理”页面安装blackhole。**  
**这个文档简单介绍了Blackhole数据分析的常用接口，更多关于blackhole使用方法和案例，请参考[Blackhole简介和基本用法](https://cloud.baidu.com/doc/BML/s/9khemrnv7)。**

# SQL引擎使用方式

下面将介绍SQL引擎基本功能的使用方式，目前SQL引擎支持python api,用户可以通过编写python程序实现和引擎的交互。

## 1.导入SQL依赖
  本教程将使用以下方式导入SQL依赖:

In [1]:
from blackhole.sql.context import Context

## 2.创建与SQL查询引擎交互的上下文环境

In [2]:
context = Context()

Current work dir is /hh/codelab/blackhole/demo/demo


## 3.建表并导入数据
我们先生成如下一份数据文件"test.csv", 数据每行包含3个字段:  
  
  
|  列名   | 说明  | 类型  |
|  ----  | ----  | ----  |
|  user | 姓名 | String |
| age | 年龄 | Int32 |
| fee  | 费用 | Float32 |

In [3]:
import os
data_path='./test.csv'
file = open(data_path, 'w+')
file.write('''Tom,32,13205.0
              Jack,31,14523.0
              Herry,28,9845.0
              Bob,43,28314.0
              Alice,23,7854.0''')
file.close()

建表前为了保证数据不受污染,首先清理可能存在的同名的表,代码如下:

In [4]:
table_name='test'
sql='drop table if exists %s' % table_name
context.sql(sql)

下面我们将使用上面代码生成的数据文件来建表并导入数据,表名称为'test',代码如下:  
* load方法从指定的文件加载数据,  
* schema方法将指定这批数据的数据结构,  
* format方法指定数据的格式,支持'CSV'、'Parquet'、'CSVWithNames'格式, 这里为“CSV”格式,  
* primary_keys方法指定数据表的主键，注意，这里主键与传统的关系数据库不同，这里pk可以重复,  
* create_table方法传入一个表名参数就可以建表了。

In [5]:
data_schema = '''user String,age Int32,fees Float32'''
data_format = 'CSV'
ds = context.load(data_path).schema(data_schema).format(data_format).primary_keys('user').create_table('test')

Start load data into table test firstly
Insert query is INSERT INTO test SELECT * FROM file('./test.csv', 'CSV', "user String,age Int32,fees Float32")


## 4.开始查询
表建好之后便可以对表中的数据进行各种查询了
  ### 4-1.全量查询表中的数据

In [6]:
context.sql('select * from test').show()

user	age	fees	
Alice	23	7854	
Bob	43	28314	
Herry	28	9845	
Jack	31	14523	
Tom	32	13205	




  ### 4-2.设置条件进行查询(过滤)

In [7]:
context.sql('select * from test where fees < 10000.0 limit 2').show()

user	age	fees	
Alice	23	7854	
Herry	28	9845	




  ### 4-3.对某列求和/求算术平均/最大/最小值

In [8]:
context.sql('select sum(fees) from test').show()
context.sql('select avg(age) from test').show()
context.sql('select min(age) from test').show()
context.sql('select max(fees) from test').show()

sum(fees)	
73741	


avg(age)	
31.4	


min(age)	
23	


max(fees)	
28314	




  ### 4-4.排序

In [9]:
context.sql('select * from test order by age desc').show()

user	age	fees	
Bob	43	28314	
Tom	32	13205	
Jack	31	14523	
Herry	28	9845	
Alice	23	7854	




  ### 4-5.聚合

In [10]:
context.sql('select user, max(fees) from test group by user').show()

user	max(fees)	
Alice	7854	
Tom	13205	
Herry	9845	
Bob	28314	
Jack	14523	




  ### 4-6.查询结果导出到文件
目前导出的文件格式为tsv(即以tab符分隔字段的文件)

In [11]:
context.sql('select age, sum(fees) from test group by age').save('./result.tsv')